In [1]:
from jax import numpy as jnp
import numpy as np
import jax
from jax.nn import sigmoid
import sys
import jax.random as random
sys.path.append("/Users/changjc/workspace/bayesianquilts")

In [ ]:
dsigmoid = lambda x: sigmoid(x) * (1 - sigmoid(x))
ddsigmoid = lambda x: sigmoid(x) * (1 - sigmoid(x)) * (1 - 2 * sigmoid(x))


#
# N x I x K


def p_ni(abilities, difficulties, discriminations):  # dimensio
    p_cum = sigmoid(discriminations * (abilities - difficulties))
    # first partials
    dp_cum_dabilities = (p_cum * (1 - p_cum))[
        ..., np.newaxis
    ] * discriminations[np.newaxis, np.newaxis, :, :, 0]
    dp_cum_ddifficulties = (
        -(p_cum * (1 - p_cum))[..., np.newaxis, np.newaxis]
        * discriminations[np.newaxis, np.newaxis, np.newaxis, ...]
    )
    dp_cum_ddiscrimintations = (p_cum * (1 - p_cum))[
        ..., np.newaxis, np.newaxis, np.newaxis
    ] * (abilities - difficulties)[np.newaxis, np.newaxis, np.newaxis, ...]

    # second partials
    # pure
    d2p_cum_dabilities2 = p_cum * (1 - p_cum) * (1 - 2 * p_cum) * discriminations**2
    d2p_cum_ddifficulties2 = (
        p_cum * (1 - p_cum) * (1 - 2 * p_cum) * discriminations * discriminations**2
    )
    d2p_cum_ddiscrimintations2 = (
        p_cum * (1 - p_cum) * (1 - 2 * p_cum) * (abilities - difficulties) ** 2
    )
    # mixed
    d2p_cum_dabilities_difficulties = (
        -p_cum * (1 - p_cum) * (1 - 2 * p_cum) * discriminations**2
    )
    d2p_cum_dabilities_discriminations = (
        p_cum * (1 - p_cum) + p_cum * (1 - p_cum) * (1 - 2 * p_cum) * discriminations**2
    )
    d2p_cum_ddifficulties_discriminations = -p_cum * (1 - p_cum) - p_cum * (
        1 - p_cum
    ) * (1 - 2 * p_cum) * discriminations * (abilities - difficulties)

    p_cum = jnp.pad(
        p_cum, ((0, 0), (0, 0), (1, 0)), constant_values=0
    )
    p_cum = jnp.pad(
        p_cum, ((0, 0), (0, 0), (0, 1)), constant_values=1
    )

    dp_cum_dabilities = jnp.pad(
        dp_cum_dabilities, ((0, 0), (0, 0), (1, 1), (0, 0), (0, 0), (0, 0)), constant_values=0
    )
    dp_cum_ddifficulties = jnp.pad(
        dp_cum_ddifficulties, ((0, 0), (0, 0), (1, 1), (0, 0), (0, 0), (0, 0)), constant_values=0
    )
    dp_cum_ddiscrimintations = jnp.pad(
        dp_cum_ddiscrimintations, ((0, 0), (0, 0), (1, 1), (0, 0), (0, 0), (0, 0)), constant_values=0
    )

    dp_dabilities = dp_cum_dabilities[:, :, 1:, ...] - dp_cum_dabilities[:, :, :-1, ...]
    dp_ddifficulties = dp_cum_ddifficulties[:, :, 1:, ...] - dp_cum_ddifficulties[:, :, :-1, ...]
    dp_ddiscrimintations = (
        dp_cum_ddiscrimintations[:, :,  1:, ...] - dp_cum_ddiscrimintations[:, :,  :-1, ...]
    )

    p = p_cum[..., 1:] - p_cum[..., :-1]

    # compute derivatives

    gradients = {
        "abilities": dp_dabilities,
        "difficulties": dp_ddifficulties,
        "discriminations": dp_ddiscrimintations,
    }

    hessians = {}

    return p, gradients, hessians

In [3]:
abilities = np.array([0, 0.5, 0.25])[:, np.newaxis, np.newaxis]
difficulties = np.array([[0, 1, 2, 3], [-2, 0, 3, 4]])[np.newaxis, ...]
discriminations = np.array([1, 2])[np.newaxis, :, np.newaxis]

In [4]:
p_ni(abilities, difficulties, discriminations)

ValueError: jnp.pad: pad_width with nd=4 has unsupported shape (6, 2). Valid shapes are (4, 2), (1, 2), (2,), (1,), or ().

In [6]:
K = 10
from autoencirt.data.rwa import item_text, get_data, to_reverse


ModuleNotFoundError: No module named 'tensorflow.io'

In [ ]:
pd_data = get_data(reorient=True, pandas=True)
X = pd_data[0].iloc[:, :22]
N, I = X.shape